In [2]:
\cd ../qenv/

In [73]:
\l test.q

`type
`[23] /home/qenv/qenv/engine/logic/testutils.q:67: .util.testutils.makeDefaultsRecords  :{[tab;cl;vl] // TODO inter with actual cols
                                                    ^
    r:.util.NullRowDict[tab];
  [20] \l /home/qenv/qenv/engine/logic/testutils.q
       ^
  [19] /home/qenv/qenv/engine/logic/init.q:14: .logic.loadTests:{
    system[sv["";("l ";x;"/testutils.q")]];
    ^
    .contract.loadTests[sv["/";(x;"contract")]];
  [18] /home/qenv/qenv/engine/init.q:11: loadTests:
    system[sv["";("l ";x;"/engineTest.q")]];
    .logic.loadTests[sv["/";(x;"logic")]];
    ^
    .valid.loadTests[sv["/";(x;"valid")]];
  [17] /home/qenv/qenv/engine/init.q:15: if[TESTING;loadTests[system["pwd"][0]]];
                                                    ^
  [14] \l engine/init.q
       ^
  [13] /home/qenv/qenv/test.q:20: 
    / \l config/init.q
    system"l engine/init.q";
    ^
    system"l state/init.q";
  [12] (.Q.trp)

  [11] /home/qenv/qenv/test.q:12: .Q.trp[{
             

In [13]:
iv:first flip .bam.x

In [27]:
.engine.model.inventory.Inventory

aId side| ordQty ordVal ordLoss amt iw mm posVal rpnl avgPrice execCost totEnt upnl lev
--------| -----------------------------------------------------------------------------
0   -1  | 0             0       10     0                                10     0       
0   1   | 0             0       10     0                                10     0       


In [17]:
.engine.logic.contract.UnrealizedPnl    :{[contractType;markPrice;faceValue;multiplier;amt;isignum;avgPrice] 
      7h$($[contractType=0;.engine.logic.contract.inverse.UnrealizedPnl[amt;isignum;avgPrice;markPrice;faceValue;multiplier]; 
            contractType=0;.engine.logic.contract.inverse.UnrealizedPnl[amt;isignum;avgPrice;markPrice;faceValue;multiplier]; 
          contractType=0;.engine.logic.contract.inverse.UnrealizedPnl[amt;isignum;avgPrice;markPrice;faceValue;multiplier]; 
      'nyi] | 0)
    };

In [16]:
.engine.logic.contract.RealizedPnl    :{[contractType;fillQty;fillPrice;isignum;avgPrice;faceValue;multiplier]
    7h$($[contractType=0;.engine.logic.contract.inverse.RealizedPnl[fillQty;fillPrice;isignum;avgPrice;faceValue;multiplier];
      contractType=1;.engine.logic.contract.linear.RealizedPnl[fillQty;fillPrice;isignum;avgPrice;faceValue;multiplier];
      contractType=2;.engine.logic.contract.quanto.RealizedPnl[fillQty;fillPrice;isignum;avgPrice;faceValue;multiplier];
      'nyi] | 0)
    };

In [47]:
select ivId.ordVal from .bam.x

ordVal|  


In [ ]:
.engine.logic.contract.ExecCost    :{[contractType;price;qty;multiplier]

fId     | 0          1         
price   | 1000       1000      
qty     | 100        100       
reduce  | 0          1         
ismaker | 0          1         
side    | 1          -1        
iId     | 0          0         
oId     | 0          1         
ivId    | 1          0         
aId     | 0          0         
avgPrice| 0          0         
execCost| 10000000   10000000  
ordLoss | 0          0         
ordVal  | -100000000 -100000000
ordQty  | -100       -100      
amt     | 110        110       
totEnt  | 210        210       


In [72]:
iv:![![.bam.x;();0b;`avgPrice`execCost`ordLoss`ordVal`ordQty`amt`totEnt!(
        ((';`.engine.logic.contract.AvgPrice);`iId.cntTyp;`side;`ivId.execCost;`ivId.totEnt;`iId.smul); // TODO dependent
        ((';`.engine.logic.contract.ExecCost);`iId.cntTyp;`price;`qty;`iId.smul);
        (-;`ivId.ordLoss;(`.engine.logic.contract.Loss;`iId.mkprice;`qty;((+/);`qty;`price)));
        (-;`ivId.ordVal;((*/);`qty;`price));
        (-;`ivId.ordQty;`qty);
        (+;`ivId.amt;`qty);
        ((+/);`ivId.totEnt;`qty)
    )];
    ();0b;`upnl`rpnl!(
    ((';`.engine.logic.contract.UnrealizedPnl);`iId.cntTyp;`iId.mkprice;`iId.faceValue;`iId.smul;`ivId.amt;`side;`ivId.avgPrice);
    ((';`.engine.logic.contract.RealizedPnl);`iId.cntTyp;`qty;`price;`side;`ivId.avgPrice;`iId.faceValue;`iId.smul))]

[0;31mnyi[0m: [0;31mnyi[0m

In [59]:
.q.cl:{x!y[x]}

In [67]:
.engine.model.inventory.Inventory upsert flip 

aId side| ordQty ordVal     ordLoss amt iw mm posVal rpnl avgPrice execCost totEnt upnl lev
--------| ---------------------------------------------------------------------------------
0   -1  | -100   -100000000 0       110    0         0    0        10000000 210    0       
0   1   | -100   -100000000 0       110    0         0    0        10000000 210    0       


In [68]:
.bam.x

fId    | 0    1   
price  | 1000 1000
qty    | 100  100 
reduce | 0    1   
ismaker| 0    1   
side   | 1    -1  
iId    | 0    0   
oId    | 0    1   
ivId   | 1    0   
aId    | 0    0   


In [ ]:
.engine.logic.inventory.Fill:{ // TODO simple select

    iv:?[?[x;();0b;(
                (`.engine.logic.contract.UnrealizedPnl;`cntTyp;`mkprice;`faceValue;`smul;`amt;`side;`avgPrice);
                (`.engine.logic.contract.RealizedPnl;`iId.cntTyp;`qty;`price;`side;`ivId.avgPrice;`iId.faceValue;`iId.smul)
        )];();0b;(
        (`.engine.logic.contract.AvgPrice);
        (`.engine.logic.contract.ExecCost);
        (-;`ivId.ordLoss;(`.engine.logic.contract.Loss;));
        (-;`ivId.ordVal;(`.engine.logic.contract.Value;));
        (-;`ivId.ordQty;qty);
        (+;`amt;`dlt);
        (sum;`ivId.totalEntry;(max;`dlt;0))
        )];

    .engine.model.inventory.Update iv;
    .engine.EmitA[`inventory;];

    		/if[abs[iv[`amt]]=0;iv[`avgPrice`execCost`totalEntry]:0];

    };



In [70]:
[(iv`kind;iv`time;iv`aId`side`time`amt`rpnl`avgPrice`upnl;iv`aId)]

`inventory
`long$()
(`.engine.model.account.Account$0 0;1 -1;`long$();110 110;0 0;0 0;0 0)
`.engine.model.account.Account$0 0


In [68]:
iv[`kind]:`inventory;
show ;
.engine.EmitA[iv`kind;s`time;flip s`aId`side`time`amt`rpnl`avgPrice`upnl;iv`aId];


[0;31mlength[0m: [0;31mlength[0m